In [56]:
import util
reload(util)
import rnnlm
reload(rnnlm)

import json, os, re, shutil, sys, time
import collections, itertools
import sys

from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("0.12"))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries
sys.path.append('shared_lib')
import utils, tf_embed_viz

print "Done!"



 Done!


In [57]:
samp = True
samp = '_samp' if samp else ''

In [64]:
# Load feature names
fnames = np.array(pd.read_pickle('../interim/028_preproc_heavy_shows_concat' + samp + '.p'))

# load concatenated descriptions
desc = np.array(pd.read_pickle('../interim/028_preproc_heavy_show_description_concat' + samp + '.p'))

print("Feature Names Shape:", fnames.shape)
print("Descriptions Shape:", desc.shape)
assert fnames.shape[0] == desc.shape[0]

('Feature Names Shape:', (1192, 2))
('Descriptions Shape:', (1192, 1))


In [59]:
reload(rnnlm)

TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)
print "Done!"

Done!


In [60]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    verbose=True
    
    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        feed_dict = { lm.input_w_: w, 
                      lm.target_y_: y, 
                      lm.initial_h_ : h, 
                      lm.learning_rate_ : learning_rate, 
                      lm.use_dropout_ : use_dropout 
                    }
        
        h, cost, _ = session.run( [lm.final_h_, loss, train_op], feed_dict=feed_dict )
        
        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [61]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=True, tick_s=3600)
    print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

In [62]:
# load the dataset
train_ids = x_train_vectors
test_ids = x_dev_vectors
vocab_size = 10000

#training parameters
max_time = 20
batch_size = 50
learning_rate = 0.5
num_epochs = 5

# Model parameters
model_params = dict(V=vocab_size, 
                    H=100, 
                    softmax_ns=200,
                    num_layers=1)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [63]:
# Will print status every this many seconds
print_interval = 15

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.batch_generator(train_ids, batch_size, max_time)
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)
        #### YOUR CODE HERE ####
        # Run a training epoch.
       
        run_epoch(lm, session, bi,
              train=True, verbose=True,
              tick_s=print_interval, learning_rate=learning_rate)
        
        #### END(YOUR CODE) ####
        print "[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, train_ids, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, test_ids, name="Test set")
        print ""
    
    # Save final model
    saver.save(session, trained_filename)

[epoch 1] Starting epoch 1
[batch 0]: seen 1000 words at 3980 wps, loss = 9.380
[batch 1]: seen 2000 words at 4557 wps, loss = 4.738
[batch 2]: seen 3000 words at 4620 wps, loss = 3.162
[batch 3]: seen 4000 words at 4642 wps, loss = 2.375
[batch 4]: seen 5000 words at 4897 wps, loss = 1.901
[batch 5]: seen 6000 words at 5091 wps, loss = 1.585
[batch 6]: seen 7000 words at 5263 wps, loss = 1.359
[batch 7]: seen 8000 words at 5389 wps, loss = 1.190
[batch 8]: seen 9000 words at 5505 wps, loss = 1.058
[batch 9]: seen 10000 words at 5533 wps, loss = 0.952
[batch 10]: seen 11000 words at 5600 wps, loss = 0.866
[batch 11]: seen 12000 words at 5670 wps, loss = 0.794
[batch 12]: seen 13000 words at 5729 wps, loss = 0.733
[batch 13]: seen 14000 words at 5783 wps, loss = 0.681
[batch 14]: seen 15000 words at 5838 wps, loss = 0.636
[batch 15]: seen 16000 words at 5884 wps, loss = 0.596
[batch 16]: seen 17000 words at 5919 wps, loss = 0.561
[batch 17]: seen 18000 words at 5943 wps, loss = 0.530
[b

u'Management & Marketing'